In [ ]:
# sort_values, sort_index : 인덱스는 키 인덱스 - 순서를 보장, 중복을 허용

In [ ]:
data['합계']=data.apply(lambda x : x.sum(),axis=1)
data

In [ ]:
data.sort_values(by=['합계'], axis=0, ascending=False)

In [ ]:
data.sort_values(by=['합계'], axis=0, ascending=False).iloc[0]

In [ ]:
data.var() # 분산

In [ ]:
data.std() # 표준편차

In [ ]:
data.iloc[:,:3].cov()

In [ ]:
data.iloc[:,:3].corr()

In [ ]:
import matplotlib.pyplot as plt

plt.matshow(data.iloc[:,:3].corr())
plt.colorbar()
plt.show()

In [ ]:
cor1 = data.iloc[:,:3].corr()
cor1.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
import seaborn as sns # matplotlib 상속받아 생성
sns.heatmap(cor1,
           xticklables=cor1.columns.values,
           yticklables=cor1.columss.values)

In [ ]:
sns.pairplot(data.iloc[:,:3])

In [ ]:
# 문제
- 데이터 베이스의 student 데이터를 가지고 와서 데이터 프레임을 초기화하고
- 성적과 총점 평균에 대하여 분산, 표준편차 합계를 구하시오

In [11]:
import pymysql
# 키의 매개변수 - 순서를 변경가능
con = pymysql.connect(host = "192.168.0.70", port=3306, user='root', passwd='20210422a',
                     db='future', charset='utf8', autocommit=True)
try:
    with con.cursor() as cur:
        cur.execute("select * from student")
        rows = cur.fetchall() # fetchone, fetchmany, fetchall
        desc = cur.description
        for row in rows: # list
            print(f'{row[0]},{row[1]}, {row[2]},{row[3]},{row[4]}') #f 함수
finally:
    con.close()
# 필드 이름을 확인
print(desc)
print([i[0] for i in cur.description])

1,김만덕, 100,100,100
2,고려인, 100,99,99
3,종로구, 100,80,70
4,전공인, 80,80,80
5,전공이, 80,80,80
7,하하, 50,80,70
12,typoon, 100,100,100
13,typoon, 100,100,100
14,융합, 100,90,80
23,별, 1,2,3
(('bunho', 3, None, 11, 11, 0, False), ('name', 253, None, 50, 50, 0, True), ('kor', 1, None, 4, 4, 0, True), ('mat', 1, None, 4, 4, 0, True), ('eng', 1, None, 4, 4, 0, True), ('total', 3, None, 11, 11, 0, True), ('average', 4, None, 12, 12, 31, True), ('grade', 254, None, 3, 3, 0, True), ('schoolcode', 254, None, 30, 30, 0, True))
['bunho', 'name', 'kor', 'mat', 'eng', 'total', 'average', 'grade', 'schoolcode']


In [16]:
import pandas as pd
student=pd.DataFrame(rows, columns=['bunho','name','kor','mat','eng','total','average','grade','schoolcode'])

In [23]:
student.set_index('bunho')

,name,kor,mat,eng,total,average,grade,schoolcode
bunho,,,,,,,,
1,김만덕,100,100,100,300,100.00,A,CH000001
2,고려인,100,99,99,298,99.33,A,CH000001
3,종로구,100,80,70,250,83.33,B,IC000001
4,전공인,80,80,80,240,80.00,B,SE000001
5,전공이,80,80,80,240,80.00,B,SE000001
7,하하,50,80,70,200,66.67,D,CH000001
12,typoon,100,100,100,300,100.00,A,CH000001
13,typoon,100,100,100,300,100.00,A,CH000001
14,융합,100,90,80,270,90.00,A,CH000001


In [20]:
student.loc[:,'kor':'average'].var()

kor        1058.766667
mat         859.211111
eng         850.844444
total      7946.488889
average     882.916646
dtype: float64

In [21]:
student.loc[:,'kor':'average'].std()

kor        32.538695
mat        29.312303
eng        29.169238
total      89.143081
average    29.713913
dtype: float64

In [24]:
student.loc[:,'kor':'average'].sum()

kor         811.00
mat         811.00
eng         782.00
total      2404.00
average     801.33
dtype: float64

In [25]:
student.loc[:,'kor':'average'].skew() # 왜도 : 정규분포에 가까운가 - 좌우대칭인가. 0에 가까울수록 좌우대칭

kor       -2.029853
mat       -2.582095
eng       -2.161609
total     -2.370767
average   -2.370842
dtype: float64

In [26]:
student.loc[:,'kor':'average'].kurt() # 첨도 3에 가까울수록 정규분포와 비슷함

kor        4.006695
mat        7.379688
eng        5.565399
total      6.275271
average    6.275707
dtype: float64

## 행렬 분해
- 정방행렬이면서 대칭행렬에 대하여 고유값 분해를 하면 고유값과 고유벡터를 출력한다.
- 이때 고유 벡터는 상호 직교한다.
- 데이터로부터 새로운 직교축을 구했다는 의미이다.
- 이때의 직교축은 데이터의 분산이 가장 큰 방향으로 축을 찾고 ( 주성분 축이다.)
- 그에 직교하는 그다음 분산이 큰 방향으로 축을 찾고
- 그 두개의 축에 직교하는 가장 분산이 큰 방향으로 축을 찾은 것이다.
- 축이 직교하면 상관계수가 0이라는 의미이다.
- 그래서 이축을 중심하고 데이터를 재표현하면 noise가 없는 데이터를 구성하게 된다.
- 재표현 된 데이터를 이용해서 분석을 하면 noise가 없는 데이터 간에 상호 영향이 없는 분석이 된다.
- 이를 주성분 분석이라고 한다.

- 행렬분해 : 계산을 효율적으로 하기 위해, 행렬의 구조를 연구하기 위해서 

- 고유값 분해는 정방행렬에만 가능, 비정방 : PCA(principle componet analysis)에 사용이 됨

- 주성분분석
- SVD(Singluar value decomposition)특이 행렬 분해 : 문자의 특징을 차원축소하기 위해

In [29]:
anal=student.loc[:,'kor':'eng']
import numpy as np
print(type(anal.corr()))
np.linalg.eig(anal.corr())

<class 'pandas.core.frame.DataFrame'>


(array([2.88092912, 0.1055947 , 0.01347618]),
 array([[ 0.56781983,  0.82040356,  0.06722079],
        [ 0.58331229, -0.34341191, -0.73608086],
        [ 0.58079894, -0.45717202,  0.67354757]]))

In [30]:
eigen_result=np.linalg.eig(anal.corr())

In [31]:
eigen_result[0]

array([2.88092912, 0.1055947 , 0.01347618])

In [32]:
eigen_result[1]

array([[ 0.56781983,  0.82040356,  0.06722079],
       [ 0.58331229, -0.34341191, -0.73608086],
       [ 0.58079894, -0.45717202,  0.67354757]])

In [33]:
eigen_result[1][1]

array([ 0.58331229, -0.34341191, -0.73608086])

In [34]:
eigen_result[1][0].dot(eigen_result[1][1])

-5.551115123125783e-17

In [36]:
eigen_result[1][:,0].dot(eigen_result[1][:,1])

-2.7755575615628914e-17

## 선형대수 : vector / matrix -> 기하공간에서 데이터 처리
- 크기와 방향

### sales.csv 로딩한 다음 다음을 해결하시오
- 1. $를 제거
- 2. %를 제거 (/100)
- 3. Y/N => True/False
- 4. 문자를 숫자로 변경할 것은 변경하시오

In [116]:
import pandas as pd
df = pd.read_csv("./sales.csv")
df

,Customer Number,Customer Name,2016,2017,Percent Growth,Jan Units,Month,Day,Year,Active
0,10002.0,Quest Industries,"$125,000.00",$162500.00,30.00%,500,1,10,2015,Y
1,552278.0,Smith Plumbing,"$920,000.00","$101,2000.00",10.00%,700,6,15,2014,Y
2,23477.0,ACME Industrial,"$50,000.00",$62500.00,25.00%,125,3,29,2016,Y
3,24900.0,Brekke LTD,"$350,000.00",$490000.00,4.00%,75,10,27,2015,Y
4,651029.0,Harbor Co,"$15,000.00",$12750.00,-15.00%,Closed,2,2,2014,N


In [117]:
df['2016']=pd.to_numeric(df['2016'].str.replace(r"[^\d]", ""))
# ==)df['2016'] = df['2016'].mpa(lambda x: x.replace('$',''))
# ==) def convert_currency(val):
#     new = val.replace(',','').repalce('$','')
#     return float(new_val)
#     def convert_precent(val):
#         new_val = val.replace('%'','')
#         return float(new_val)/100
#  apply(행,열), applymap(Series), map(개별데이터에 적용)

df['2017']=pd.to_numeric(df['2017'].str.replace(r"[^\d]", ""))
df['Percent Growth']=pd.to_numeric(df['Percent Growth'].str.replace(r"[$%]", ""))
df

,Customer Number,Customer Name,2016,2017,Percent Growth,Jan Units,Month,Day,Year,Active
0,10002.0,Quest Industries,12500000,16250000,30.0,500,1,10,2015,Y
1,552278.0,Smith Plumbing,92000000,101200000,10.0,700,6,15,2014,Y
2,23477.0,ACME Industrial,5000000,6250000,25.0,125,3,29,2016,Y
3,24900.0,Brekke LTD,35000000,49000000,4.0,75,10,27,2015,Y
4,651029.0,Harbor Co,1500000,1275000,-15.0,Closed,2,2,2014,N


In [123]:
df.iloc[:,2:5].astype(int)
df['Customer Number'] = df['Customer Number'].astype(int)

In [119]:
df['Active']=df['Active'].apply(lambda x: 'True' if x == 'Y' else 'False')

In [120]:
df['Percent Growth']=df['Percent Growth']/100

In [126]:
df['Jan Units'] = pd.to_numeric(df['Jan Units'], errors='coerce')
# raise 에러 발생시켜라 defalut
# coerce nan 공백으로 처리
# ignore 무시

In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Customer Number  5 non-null      int32  
 1   Customer Name    5 non-null      object 
 2   2016             5 non-null      int64  
 3   2017             5 non-null      int64  
 4   Percent Growth   5 non-null      float64
 5   Jan Units        4 non-null      float64
 6   Month            5 non-null      int64  
 7   Day              5 non-null      int64  
 8   Year             5 non-null      int64  
 9   Active           5 non-null      object 
dtypes: float64(2), int32(1), int64(5), object(2)
memory usage: 508.0+ bytes


In [128]:
df

,Customer Number,Customer Name,2016,2017,Percent Growth,Jan Units,Month,Day,Year,Active
0,10002,Quest Industries,12500000,16250000,0.30,500.0,1,10,2015,True
1,552278,Smith Plumbing,92000000,101200000,0.10,700.0,6,15,2014,True
2,23477,ACME Industrial,5000000,6250000,0.25,125.0,3,29,2016,True
3,24900,Brekke LTD,35000000,49000000,0.04,75.0,10,27,2015,True
4,651029,Harbor Co,1500000,1275000,-0.15,NaN,2,2,2014,False


In [139]:
def convert_currency(val):
    new_val = val.replace(',','').replace('$','')
    return float(new_val)
def convert_precent(val):
    new_val = val.replace('%','')
    return float(new_val)/100
df_2 = pd.read_csv("sales.csv",
                  dtype={'Customer Number' : 'int'},
                  converters={'2016':convert_currency,
                  '2016':convert_currency,
                    'Jan Units': lambda x: pd.to_numeric(x, errors='coerce'),
                    'Active': lambda x: np.where(x == "Y", True, False)
})

In [140]:
df_2

,Customer Number,Customer Name,2016,2017,Percent Growth,Jan Units,Month,Day,Year,Active
0,10002,Quest Industries,125000.0,$162500.00,30.00%,500.0,1,10,2015,True
1,552278,Smith Plumbing,920000.0,"$101,2000.00",10.00%,700.0,6,15,2014,True
2,23477,ACME Industrial,50000.0,$62500.00,25.00%,125.0,3,29,2016,True
3,24900,Brekke LTD,350000.0,$490000.00,4.00%,75.0,10,27,2015,True
4,651029,Harbor Co,15000.0,$12750.00,-15.00%,NaN,2,2,2014,False


In [194]:
# - gender 열과 resident열을 출력하시오
# - price 는 평균, age는 합계를 계산하시오
dataset=pd.read_csv("dataset.csv")

In [195]:
dataset.iloc[:,0:2]

,resident,gender
0,1,1
1,2,1
2,,1
3,4,2
4,5,1
...,...,...
295,1,5
296,3,1
297,4,1
298,3,0


In [209]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   resident  300 non-null    object 
 1   gender    300 non-null    int64  
 2   job       300 non-null    object 
 3   age       300 non-null    object 
 4   position  300 non-null    object 
 5   price     270 non-null    float64
 6   survey    300 non-null    int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 16.5+ KB


In [197]:
dataset.iloc[:,5]=dataset.iloc[:,5].str.strip()

In [176]:
dataset.iloc[:,5].unique()

array(['5.10', '4.20', '4.70', '3.50', '5.00', '5.40', '4.10', '675.00',
       '4.40', '4.90', '2.30', '6.70', '4.30', '257.80', '5.70', '4.60',
       '2.10', '6.20', '-75.00', '', '5.20', '6.30', '4.00', '5.30',
       '7.10', '85.10', '3.80', '7.00', '6.40', '5.50', '3.90', '6.00',
       '3.30', '-457.20', '5.60', '-5.90', '3.40', '3.00', '6.80', '6.10',
       '5.80', '115.70', '75.10', '6.50', '7.20', '7.70', '336.50',
       '5.90', '225.80', '-4.80', '4.80', '-345.60', '6.90', '-235.80',
       '7.90', '4.50'], dtype=object)

In [198]:
dataset.columns=dataset.columns.str.strip()

In [201]:
dataset["price"].fillna('',inplace=True)

In [202]:
dataset['price']=pd.to_numeric(dataset['price'], errors='ignore')

In [204]:
dataset['price'].mean()

8.75148148148148

In [211]:
dataset.age=dataset.age.str.strip()
dataset['age']=pd.to_numeric(dataset['age'], errors='ignore')

In [213]:
dataset.age.sum()

11651.0

In [30]:
import requests # 웹 브라우저 처럼 = > text(tag+javascript)
import pandas as pd
from bs4 import BeautifulSoup # tag파싱 => DOM (계층적으로 노드를 구성) : select , find
import re # regular expression 정규표현식 : 패턴으로 인식
class HTMLParser: #테이블을 동시에 파싱
    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        return [(table('id'), self.parse_html_table(table))
                   for table in soup.find_all('table')]
    def parse_html_table(self, table):
        n_columns = 0
        n_rows=0
        column_names = [] # 열이름
        for row in table.find_all('tr'): # 행
            td_tags = row.find_all('td') # 열
            #print(row)
            if len(td_tags) > 0: # 행이 있으면
                n_rows+=1
                if n_columns == 0: # 컬럼을 초기화
                    n_columns = len(td_tags)
            th_tags = row.find_all('th') # 열이름
            if len(th_tags) > 0 and len(column_names) == 0: # 열이름이 결정이 안되었으면
                for th in th_tags:
                    column_names.append(th.get_text()) # tag + 속성 + 문자열
            #print(n_columns)
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("컬럼 타이틀이 컬럼 숫자와 맞지 않는다.")
            # 열이름이 있으면 열이름을 사용하고 아니면 숫자 인덱스를 사용
        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        print(columns)
        df = pd.DataFrame(columns = columns,index =  range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            #print(row)
            column_marker = 0
            columns = row.find_all('td')
            for column in columns: #1, 2, 3
                re_text=column.get_text().replace("\n",'')
                re_text=re_text.replace("\t",'')
                re_text=re_text.replace("\r",'')
                df.iloc[row_marker,column_marker] = re_text
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1
        for col in df:
            try:
                df[col] = df[col].astype(float) # 전체데이터를 숫자로 
            except ValueError:
                pass
        return df

In [31]:
url = "https://www.w3schools.com/html/html_tables.asp"
print(url)
hp = HTMLParser()
table = hp.parse_url(url)[0][1]
table

https://www.w3schools.com/html/html_tables.asp
['Company', 'Contact', 'Country']
['Tag', 'Description']


,Company,Contact,Country
0,Alfreds Futterkiste,Maria Anders,Germany
1,Centro comercial Moctezuma,Francisco Chang,Mexico
2,Ernst Handel,Roland Mendel,Austria
3,Island Trading,Helen Bennett,UK
4,Laughing Bacchus Winecellars,Yoshi Tannamuri,Canada
5,Magazzini Alimentari Riuniti,Giovanni Rovelli,Italy


In [33]:
data = "2021.04.02"
url_part1 = """https://dart.fss.or.kr/dsac001/search.ax?selectData="""
url_part2 = """&sort=&series=&mdayCnt=0&currentPage="""

url = url_part1 + data + url_part2
print(url)
hp = HTMLParser()
table = hp.parse_url(url)[0][1]
table

https://dart.fss.or.kr/dsac001/search.ax?selectData=2021.04.02&sort=&series=&mdayCnt=0&currentPage=
['시간', '공시대상회사', '보고서명', '제출인', '접수일자', '비고']


,시간,공시대상회사,보고서명,제출인,접수일자,비고
0,16:35,에스크컴퍼니,연결감사보고서 (2020.12),신한회계법인,2021.04.30,
1,16:34,와이제이코퍼레이션,연결감사보고서 (2020.12),우덕회계법인,2021.04.30,
2,16:34,대림제7호마산회원기업형임대위탁관리부동산투자회사,임원의변동,대림제7호마산회원기업형임대위탁관리부동산투자회사,2021.04.30,
3,16:34,애드바이오텍,사업보고서 (2020.12),애드바이오텍,2021.04.30,
4,16:34,삼성생명,최대주주변경,삼성생명,2021.04.30,
...,...,...,...,...,...,...
95,16:08,DL이앤씨,결산실적공시예고(안내공시),DL이앤씨,2021.04.30,
96,16:07,덕일산업,연결감사보고서 (2020.12),인덕회계법인,2021.04.30,
97,16:07,동아전장,연결감사보고서 (2020.12),우리회계법인,2021.04.30,
98,16:07,이화전기,[기재정정]전환청구권행사,이화전기,2021.04.30,


In [ ]:
# 불필요한 데이터를 제거하고 정상적인 페이지의 내용으로 출력해 보시오

In [ ]:
# 마지막 : 새로운 table을 가진 페이지에서 테이블 파싱해 보시오
=> DB에 저장하고
=> 저장된 데이터를 DataFrame으로 초기화
=> 분석후 시각화하는 그래프를 출력한 것을 제출

In [36]:
url = "https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx"
print(url)
hp = HTMLParser()
table = hp.parse_url(url)[0][1]

https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx
['순위', '선수명', '팀명', 'AVG', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'TB', 'RBI', 'SAC', 'SF']


In [38]:
table

,순위,선수명,팀명,AVG,G,PA,AB,R,H,2B,3B,HR,TB,RBI,SAC,SF
0,1.0,강민호,삼성,0.400,21.0,82.0,75.0,10.0,30.0,4.0,0.0,3.0,43.0,17.0,0.0,1.0
1,2.0,강백호,KT,0.393,22.0,102.0,89.0,9.0,35.0,4.0,1.0,2.0,47.0,19.0,0.0,3.0
2,3.0,박건우,두산,0.373,22.0,90.0,75.0,15.0,28.0,4.0,0.0,2.0,38.0,15.0,0.0,2.0
3,4.0,최주환,SSG,0.365,19.0,79.0,74.0,8.0,27.0,6.0,0.0,4.0,45.0,15.0,0.0,0.0
4,5.0,허경민,두산,0.362,22.0,101.0,94.0,17.0,34.0,4.0,0.0,1.0,41.0,8.0,0.0,0.0
5,6.0,알테어,NC,0.346,22.0,90.0,81.0,15.0,28.0,5.0,0.0,9.0,60.0,22.0,0.0,1.0
6,7.0,구자욱,삼성,0.345,23.0,101.0,84.0,18.0,29.0,7.0,1.0,4.0,50.0,19.0,0.0,2.0
7,8.0,피렐라,삼성,0.337,23.0,101.0,92.0,17.0,31.0,3.0,1.0,8.0,60.0,18.0,0.0,1.0
8,9.0,서건창,키움,0.333,22.0,90.0,75.0,14.0,25.0,5.0,0.0,1.0,33.0,7.0,0.0,0.0
9,10.0,홍창기,LG,0.329,22.0,101.0,82.0,15.0,27.0,4.0,1.0,1.0,36.0,11.0,0.0,1.0


In [ ]:
df.to_sql(name='db의 테이블이름', con=db_connection, if_exists='append',index=False) 